In [34]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

* ~~Better metric for evaulation. Percent correct~~
* ~~Consider applying time series data to create new features(last games runs, 5 game average runs)~~
* Better features like team batting average, starting pitcher ERA
* Normalize/Standardize features
    * https://www.geeksforgeeks.org/normalization-vs-standardization/
* Consider applying a Deep Learning Model
    * https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/regression.ipynb
* Try bagging: BaggingRegressor
* Consider Hyperparameter tuning using Hyperopt

In [35]:
import os 
# data = pd.read_csv(os.path.join("/kaggle/input/mlb-teams-runs-20122021-no-ohe-no-date/allTeams_data.csv"))
# data = pd.read_csv(os.path.join("/kaggle/input/ml-20122021-no-outliers-ohe-teams/allTeams_data.csv"))
# data = pd.read_csv(os.path.join("/kaggle/input/mlb-teams-runs-20122021-no-outliers-ohe-teams-r1/allTeams_data.csv"))
# data = pd.read_csv(os.path.join("/kaggle/input/mlb-teams-runs-200122021-no-outliers-ohe-r1-r3-r5/allTeams_data.csv"))
# data = pd.read_csv(os.path.join("/kaggle/input/mlb-teams-runs-20122021-no-outliers-ohe-r1r10/allTeams_data.csv"))
# data = pd.read_csv(os.path.join("/kaggle/input/mlb-teams-runs-20022021-no-out-ohe-r1r10/allTeams_data.csv"))


# data = pd.read_csv(os.path.join("/kaggle/input/mlb-teams-runs-20022021-no-outliers-ohe-teams/allTeams_data.csv"))
# data = pd.read_csv(os.path.join("/kaggle/input/mlb-teams-runs-20122021-no-outliers/allTeams_data.csv"))
# data = pd.read_csv(os.path.join("/kaggle/input/mlb-teams-runs-20122021/allTeams_data.csv"))
# data = pd.read_csv(os.path.join("/kaggle/input/mlb-teams-runs-20022021/allTeams_data.csv"))

# data = pd.read_csv(os.path.join("/kaggle/input/mlb-teams-runs-20022021-no-out-ohe-r1r10-dates/allTeams_data.csv"))

data = pd.read_csv('allTeams_data.csv')

X = data.drop(columns=["R", "D/N", "Streak", "Rank", "GB"])
# X = data.drop(columns=["R"])
Y = data["R"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

### Linear Regression

In [36]:
# parameters = {'fit_intercept': [True, False], 'normalize': [True, False]}
# models = GridSearchCV(LinearRegression(), parameters, verbose=1)
# models.fit(X_train, y_train)
# models.best_params_

In [37]:
# scoresLR = cross_val_score(models.best_estimator_, X, Y, scoring='neg_mean_squared_error')
# np.mean((-scoresLR)) 

### Random Forest

In [38]:
parameters2 = {'n_estimators': list(range(100, 180, 5)), 'max_leaf_nodes': list(range(80, 150, 10)), 'min_samples_split': [10, 11, 12, 13, 14, 15], 'max_features': ["auto"]}
# parameters2 = {'n_estimators': list(range(80, 120, 10)), 'max_leaf_nodes': list(range(50, 100, 10)), 'min_samples_split': [8, 9, 10], 'max_features': ["auto"]}
models2 = GridSearchCV(RandomForestRegressor(), parameters2, verbose=1)
models2.fit(X_train, y_train)
models2.best_params_

Fitting 5 folds for each of 672 candidates, totalling 3360 fits


In [ ]:
scoresRF = cross_val_score(models2.best_estimator_, X, Y, scoring='neg_mean_squared_error')
np.mean((-scoresRF)**0.5)

### XGBoost

In [ ]:
parameters3 = {
    'nthread':[4], #when use hyperthread, xgboost may become slower
    'learning_rate': [0.05, 0.06, 0.07, 0.08], #so called `eta` value
    'n_estimators': [80, 90, 100, 110, 120, 130, 140, 150],
    'min_child_weight': [70, 80, 90, 100],
    'gamma': [1.5],
    'subsample': [0.8, 0.9, 0.95],
    'colsample_bytree': [1.0],
    'max_depth': [3, 4, 5, 6, 7, 8]
}
# parameters3 = {
#     'nthread':[4], #when use hyperthread, xgboost may become slower
#     'learning_rate': [0.06, 0.07], #so called `eta` value
#     'n_estimators': [70, 80, 90],
#     'min_child_weight': [70, 80, 90],
#     'gamma': [1.5],
#     'subsample': [0.8, 0.9],
#     'colsample_bytree': [1.0],
#     'max_depth': [5, 6, 7]
# }
models3 = GridSearchCV(XGBRegressor(), parameters3, verbose=1)
models3.fit(X_train, y_train)
models3.best_params_

In [ ]:
scoresXG = cross_val_score(models3.best_estimator_, X, Y, scoring='neg_mean_squared_error')
np.mean((-scoresXG)**0.5) 

In [ ]:
bestModel = XGBRegressor(colsample_bytree=1.0,gamma=1.5, learning_rate=0.06,max_depth=6,min_child_weight=70,
                         n_estimators=90, nthread=4, subsample=0.9)
bestModel = RandomForestRegressor(n_estimators=110, 
                                  max_leaf_nodes=90, 
                                  min_samples_split=10)
bestModel.fit(X_train, y_train)
results = bestModel.predict(X_test)
# results = bestModel.predict(X_train)

scoresXG = cross_val_score(bestModel, X_train, y_train, scoring='neg_mean_squared_error')
np.mean((-scoresXG)**0.5)  

In [ ]:
print(bestModel.feature_importances_)
print(X.columns)

In [ ]:
resultsRound = np.around(results)
resultsRound = resultsRound.astype(int)
correct = resultsRound == y_test
print("Percent Correct (rounded results): " + str(correct.sum()/len(y_test)))
print("MSE: " + str(mean_squared_error(y_test, resultsRound)))

In [ ]:
correct5 = 0
correct10 = 0
correct15 = 0
correct20 = 0
correct25 = 0
correct30 = 0
correct35 = 0
correct40 = 0
correct45 = 0
correct50 = 0
for r, y in zip(results, y_test):
    if (y >= r - .5) and (y <= r + .5):
        correct5 += 1
    if (y >= r - 1) and (y <= r + 1):
        correct10 += 1
    if (y >= r - 1.5) and (y <= r + 1.5):
        correct15 += 1
    if (y >= r - 2) and (y <= r + 2):
        correct20 += 1
    if (y >= r - 2.5) and (y <= r + 2.5):
        correct25 += 1
    if (y >= r - 3) and (y <= r + 3):
        correct30 += 1
    if (y >= r - 3.5) and (y <= r + 3.5):
        correct35 += 1
    if (y >= r - 4) and (y <= r + 4):
        correct40 += 1
    if (y >= r - 4.5) and (y <= r + 4.5):
        correct45 += 1
    if (y >= r - 5) and (y <= r + 5):
        correct50 += 1
        
print("Percent Correct within .5: " + str(correct5/len(y_test)))
print("Percent Correct within 1: " + str(correct10/len(y_test)))
print("Percent Correct within 1.5: " + str(correct15/len(y_test)))
print("Percent Correct within 2: " + str(correct20/len(y_test)))
print("Percent Correct within 2.5: " + str(correct25/len(y_test)))
print("Percent Correct within 3: " + str(correct30/len(y_test)))
print("Percent Correct within 3.5: " + str(correct35/len(y_test)))
print("Percent Correct within 4: " + str(correct40/len(y_test)))
print("Percent Correct within 4.5: " + str(correct45/len(y_test)))
print("Percent Correct within 5: " + str(correct50/len(y_test)))

In [ ]:
correct5 = 0
correct10 = 0
correct20 = 0
correct30 = 0
correct40 = 0
correct50 = 0
for r, y in zip(resultsRound, y_test):
    if y == r:
        correct5 += 1
    if (y >= r - 1) and (y <= r + 1):
        correct10 += 1
    if (y >= r - 2) and (y <= r + 2):
        correct20 += 1
    if (y >= r - 3) and (y <= r + 3):
        correct30 += 1
    if (y >= r - 4) and (y <= r + 4):
        correct40 += 1
    if (y >= r - 5) and (y <= r + 5):
        correct50 += 1
        
print("Percent Correct (rounded results): " + str(correct5/len(y_test)))
print("Percent Correct within 1: " + str(correct10/len(y_test)))
print("Percent Correct within 2: " + str(correct20/len(y_test)))
print("Percent Correct within 3: " + str(correct30/len(y_test)))
print("Percent Correct within 4: " + str(correct40/len(y_test)))
print("Percent Correct within 5: " + str(correct50/len(y_test)))